In [49]:
import pandas as pd
import numpy as np

In [50]:
df = pd.read_csv(r'DDW-C19-0000.csv') # C-19 POPULATION BY BILINGUALISM, TRILINGUALISM, EDUCATIONAL LEVEL AND SEX
df = df[5:] # STRIPPING UNWANTED ROWS
df = df[:-3] # STRIPPING LAST UNWANTED ROWS AS WELL
df.drop(df.columns[[1, 5, 6, 7, 8]], axis = 1, inplace = True) # DROPPING UNWANTED COLUMNS
# RENAMING COLUMNS WITH SUITABLE NAMES
df.columns = ['state/ut','AreaName','Total/Rural/Urban','literacy-group','Males-Numberspeakingthirdlanguage','Females-Numberspeakingthirdlanguage']

In [51]:
# EXTRACTING ONLY REQUIRED ROWS AND THEN DROPPING UNWANTED COLUMNS
df = df[df['Total/Rural/Urban']=="Total"]
df = df.drop(['Total/Rural/Urban'], axis = 1)
df = df.drop(df[df['literacy-group']=="Total"].index)

In [52]:
dfp = pd.read_csv(r'DDW-0000C-08.csv') # C-8  EDUCATIONAL LEVEL BY AGE AND SEX FOR POPULATION AGE 7 AND ABOVE - 2011
dfp = dfp[6:]   # STRIPPING UNWANTED ROWS
# DROPPING UNWANTED COLUMNS
dfp.drop(dfp.columns[[0,2,6,7,8,9,12,15,18,21,24,27,30,33,36,39,42,43,44,45]], axis = 1, inplace = True)
# RENAMING COLUMNS WITH SUITABLE NAMES
dfp.columns = ['state/ut','state-name','Total/Rural/Urban','literacy-group','Illiterate-males','Illiterate-females','Literate-males','Literate-females','Literate_wel-males', 'Literate_wel-females', 'Literate but below primary-males', 'Literate but below primary-females', 'Primary but below middle-males', 'Primary but below middle-females', 'Middle but below matric/secondary-males', 'Middle but below matric/secondary-females', 'Matric/Secondary but below graduate-males', 'Matric/Secondary but below graduate-females', 'Higher_secondary-males', 'Higher_secondary-females', 'Non_technical_diploma-males', 'Non_technical_diploma-females', 'Technical_diploma-males', 'Technical_diploma-females', 'Graduate and above-males', 'Graduate and above-females']

In [53]:
# EXTRACTING ONLY REQUIRED ROWS AND THEN DROPPING UNWANTED COLUMNS
dfp = dfp[dfp['Total/Rural/Urban']=="Total"]
dfp = dfp.drop(['Total/Rural/Urban'], axis = 1)
dfp = dfp[dfp['literacy-group']=="All ages"]
dfp = dfp.drop(['literacy-group'], axis = 1)
# CONVERTING REQUIRED COLUMNS INTO SPECIFIC DATA TYPE
dfp = dfp.astype({'Illiterate-males' :float,'Illiterate-females' :float,'Literate-males' :float,'Literate-females' :float,'Literate_wel-males' :float,'Literate_wel-females' :float,'Literate but below primary-males' :float,'Literate but below primary-females' :float,'Primary but below middle-males' :float,'Primary but below middle-females' :float,'Middle but below matric/secondary-males' :float,'Middle but below matric/secondary-females' :float,'Matric/Secondary but below graduate-males' :float,'Matric/Secondary but below graduate-females' :float,'Higher_secondary-males' :float,'Higher_secondary-females' :float,'Non_technical_diploma-males' :float,'Non_technical_diploma-females' :float,'Technical_diploma-males' :float,'Technical_diploma-females' :float,'Graduate and above-males' :float,'Graduate and above-females' :float})

#dfp['Literate-males'] = dfp['Literate-males']+dfp['Literate_wel-males']
#dfp['Literate-females'] = dfp['Literate-females']+dfp['Literate_wel-females']
#dfp = dfp.drop(['Literate_wel-males'], axis = 1)
#dfp = dfp.drop(['Literate_wel-females'], axis = 1)

dfp['Literate but below primary-males'] = dfp['Literate but below primary-males']+dfp['Literate_wel-males']
dfp['Literate but below primary-females'] = dfp['Literate but below primary-females']+dfp['Literate_wel-females']
dfp = dfp.drop(['Literate_wel-males','Literate-males'], axis = 1)
dfp = dfp.drop(['Literate_wel-females','Literate-females'], axis = 1)


dfp['Matric/Secondary but below graduate-males'] = dfp['Matric/Secondary but below graduate-males']+dfp['Higher_secondary-males']+dfp['Non_technical_diploma-males']+dfp['Technical_diploma-males']
dfp['Matric/Secondary but below graduate-females'] = dfp['Matric/Secondary but below graduate-females']+dfp['Higher_secondary-females']+dfp['Non_technical_diploma-females']+dfp['Technical_diploma-females']
# DROPPING UNWANTED COLUMNS
dfp = dfp.drop(['Higher_secondary-males','Non_technical_diploma-males','Technical_diploma-males'], axis = 1)
dfp = dfp.drop(['Higher_secondary-females','Non_technical_diploma-females','Technical_diploma-females'], axis = 1)

In [54]:
# CONVERTING COLUMNS TO ROWS BY MELT FUNCTION JUST TO MAKE OUR MERGING PROCESS EASY
dfp = dfp.melt(id_vars=["state/ut", "state-name"], var_name="literacy-group", value_name="Total persons")

In [55]:
# ADDING SEPARATE GENDER COLUMN INTO DATAFRAME
dfp['gender'] = ['males' if i.find('-males')!=-1  else 'females' for i in dfp['literacy-group']]

In [56]:
# NOTE: THESE POPULATION DATAFRAMES ARE ADJUSTED TO SPECIFIC AGE INTERVALS THAT WE ARE WORKING UPON, THIS IS DONE ONLY ONCE
# AND HERE, SO WE ARE GONNA REUSE THE POPULATION DATAFRAMES i.e dfp_males,dfp_females IN LATER PARTS WITHOUT COMPUTING AGAIN AND AGAIN.
# DIVIDING MALES AND FEMALES INTO SEPARATE DATAFRAMES AND REMOVING UNWANTED COLUMNS
dfp_males = dfp[dfp['gender']=='males']
dfp_males['literacy-group'] = [i[:-6] for i in dfp_males['literacy-group']]
dfp_males.rename(columns = {'Total persons':'males-total'}, inplace = True)
dfp_males = dfp_males.drop(['state-name','gender'], axis = 1)
dfp_females = dfp[dfp['gender']=='females']
dfp_females['literacy-group'] = [i[:-8] for i in dfp_females['literacy-group']]
dfp_females.rename(columns = {'Total persons':'females-total'}, inplace = True)
dfp_females = dfp_females.drop(['state-name','gender'], axis = 1)

C:\Users\siddh\AppData\Local\Temp/ipykernel_13760/1868803875.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfp_males['literacy-group'] = [i[:-6] for i in dfp_males['literacy-group']]
C:\Users\siddh\AppData\Local\Temp/ipykernel_13760/1868803875.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfp_females['literacy-group'] = [i[:-8] for i in dfp_females['literacy-group']]


In [57]:
# MERGING DATA FRAMES ONE BY ONE
merged_df = df.merge(dfp_males,on=["state/ut","literacy-group"])
merged_df = merged_df.merge(dfp_females,on=["state/ut","literacy-group"])

In [58]:
# CONVERTING REQUIRED COLUMNS INTO SPECIFIC DATA TYPE
merged_df = merged_df.astype({'Males-Numberspeakingthirdlanguage':float,'Females-Numberspeakingthirdlanguage':float,'males-total':float,'females-total':float})
merged_df['males-ratio-of-3'] = merged_df['Males-Numberspeakingthirdlanguage']/merged_df['males-total']
merged_df['females-ratio-of-3'] = merged_df['Females-Numberspeakingthirdlanguage']/merged_df['females-total']

In [59]:
# Finding literacy group separately for males and females that has the highest ratio of population that can speak 3 or more languages
final_df = pd.DataFrame(columns = ['state/ut','literacy-group-males','ratio-males','literacy-group-females','ratio-females'])
for i in merged_df['state/ut'].unique():
    dfi = merged_df[merged_df['state/ut']==i]
    dfi = dfi.astype({'males-ratio-of-3':float})
    dfi = dfi.astype({'females-ratio-of-3':float})
    x = dfi.loc[dfi['males-ratio-of-3'].idxmax()]
    y = dfi.loc[dfi['females-ratio-of-3'].idxmax()]
    final_df = final_df.append(pd.DataFrame({"state/ut": i,"literacy-group-males":x['literacy-group'],'ratio-males':x['males-ratio-of-3'],"literacy-group-females":y['literacy-group'],'ratio-females':y['females-ratio-of-3']}, index=[0]))

In [60]:
final_df.to_csv(r'literacy-gender-a.csv', index = None, header=True) # WRITING OUR DATAFRAME INTO OUTPUT CSV FILE

In [61]:
df = pd.read_csv(r'DDW-C19-0000.csv') # C-19 POPULATION BY BILINGUALISM, TRILINGUALISM, EDUCATIONAL LEVEL AND SEX
df = df[5:] # STRIPPING UNWANTED ROWS
df = df[:-3] # STRIPPING UNWANTED ROWS AT THE END
df.drop(df.columns[[1, 5, 8]], axis = 1, inplace = True) # DROPPING UNWANTED COLUMNS
# RENAMING COLUMNS WITH SUITABLE NAMES
df.columns = ['state/ut','AreaName','Total/Rural/Urban','literacy-group','Males-Numberspeakingsecondlanguage','Females-Numberspeakingsecondlanguage','Males-Numberspeakingthirdlanguage','Females-Numberspeakingthirdlanguage']
# CONVERTING REQUIRED COLUMNS INTO SPECIFIC DATA TYPE
df = df.astype({'Males-Numberspeakingsecondlanguage':float,'Males-Numberspeakingthirdlanguage':float,'Females-Numberspeakingsecondlanguage':float,'Females-Numberspeakingthirdlanguage':float})
df['Males-Numberspeakingsecondlanguage'] = df['Males-Numberspeakingsecondlanguage']-df['Males-Numberspeakingthirdlanguage']
df['Females-Numberspeakingsecondlanguage'] = df['Females-Numberspeakingsecondlanguage']-df['Females-Numberspeakingthirdlanguage']
df.drop(df.columns[[6,7]], axis = 1, inplace = True) # DROPPING UNWANTED COLUMNS

In [62]:
# EXTRACTING ONLY REQUIRED ROWS AND THEN DROPPING UNWANTED COLUMNS
df = df[df['Total/Rural/Urban']=="Total"]
df = df.drop(['Total/Rural/Urban'], axis = 1)
df = df.drop(df[df['literacy-group']=="Total"].index)

In [63]:
# MERGING DATA FRAMES ONE BY ONE
merged_df = df.merge(dfp_males,on=["state/ut","literacy-group"])
merged_df = merged_df.merge(dfp_females,on=["state/ut","literacy-group"])

In [64]:
# CONVERTING REQUIRED COLUMNS INTO SPECIFIC DATA TYPE
merged_df = merged_df.astype({'Males-Numberspeakingsecondlanguage':float,'Females-Numberspeakingsecondlanguage':float,'males-total':float,'females-total':float})
merged_df['males-ratio-of-2'] = merged_df['Males-Numberspeakingsecondlanguage']/merged_df['males-total']
merged_df['females-ratio-of-2'] = merged_df['Females-Numberspeakingsecondlanguage']/merged_df['females-total']

In [65]:
# Finding literacy group separately for males and females that has the highest ratio of population that can speak exactly 2 languages
final_df = pd.DataFrame(columns = ['state/ut','literacy-group-males','ratio-males','literacy-group-females','ratio-females'])
for i in merged_df['state/ut'].unique():
    dfi = merged_df[merged_df['state/ut']==i]
    dfi = dfi.astype({'males-ratio-of-2':float})
    dfi = dfi.astype({'females-ratio-of-2':float})
    x = dfi.loc[dfi['males-ratio-of-2'].idxmax()]
    y = dfi.loc[dfi['females-ratio-of-2'].idxmax()]
    final_df = final_df.append(pd.DataFrame({"state/ut": i,"literacy-group-males":x['literacy-group'],'ratio-males':x['males-ratio-of-2'],"literacy-group-females":y['literacy-group'],'ratio-females':y['females-ratio-of-2']}, index=[0]))

In [66]:
final_df.to_csv(r'literacy-gender-b.csv', index = None, header=True) # WRITING OUR DATAFRAME INTO OUTPUT CSV FILE

In [67]:
df = pd.read_csv(r'DDW-C19-0000.csv') # C-19 POPULATION BY BILINGUALISM, TRILINGUALISM, EDUCATIONAL LEVEL AND SEX
df = df[5:] # STRIPPING UNWANTED ROWS
df = df[:-3]  # STRIPPING UNWANTED ROWS AT THE END
df.drop(df.columns[[1, 5, 8]], axis = 1, inplace = True) # DROPPING UNWANTED COLUMNS
# RENAMING COLUMNS WITH SUITABLE NAMES
df.columns = ['state/ut','AreaName','Total/Rural/Urban','literacy-group','Males-Numberspeakingsecondlanguage','Females-Numberspeakingsecondlanguage','Males-Numberspeakingthirdlanguage','Females-Numberspeakingthirdlanguage']
df.drop(df.columns[[6,7]], axis = 1, inplace = True) # DROPPING UNWANTED COLUMNS

In [68]:
# EXTRACTING ONLY REQUIRED ROWS AND THEN DROPPING UNWANTED COLUMNS
df = df[df['Total/Rural/Urban']=="Total"]
df = df.drop(['Total/Rural/Urban'], axis = 1)
df = df.drop(df[df['literacy-group']=="Total"].index)

In [69]:
# MERGING DATA FRAMES ONE BY ONE
merged_df = df.merge(dfp_males,on=["state/ut","literacy-group"])
merged_df = merged_df.merge(dfp_females,on=["state/ut","literacy-group"])

In [70]:
# CONVERTING REQUIRED COLUMNS INTO SPECIFIC DATA TYPE
merged_df = merged_df.astype({'Males-Numberspeakingsecondlanguage':float,'Females-Numberspeakingsecondlanguage':float,'males-total':float,'females-total':float})
merged_df['Males-Numberspeakingfirstlanguage'] = merged_df['males-total']-merged_df['Males-Numberspeakingsecondlanguage']
merged_df['Females-Numberspeakingfirstlanguage'] = merged_df['females-total']-merged_df['Females-Numberspeakingsecondlanguage']
merged_df = merged_df.drop(['Males-Numberspeakingsecondlanguage','Females-Numberspeakingsecondlanguage'], axis = 1)
merged_df['males-ratio-of-1'] = merged_df['Males-Numberspeakingfirstlanguage']/merged_df['males-total']
merged_df['females-ratio-of-1'] = merged_df['Females-Numberspeakingfirstlanguage']/merged_df['females-total']

In [71]:
# Finding literacy group separately for males and females that has the highest ratio of population that can speak exactly 1 language
final_df = pd.DataFrame(columns =  ['state/ut','literacy-group-males','ratio-males','literacy-group-females','ratio-females'])
for i in merged_df['state/ut'].unique():
    dfi = merged_df[merged_df['state/ut']==i]
    dfi = dfi.astype({'males-ratio-of-1':float})
    dfi = dfi.astype({'females-ratio-of-1':float})
    x = dfi.loc[dfi['males-ratio-of-1'].idxmax()]
    y = dfi.loc[dfi['females-ratio-of-1'].idxmax()]
    final_df = final_df.append(pd.DataFrame({"state/ut": i,"literacy-group-males":x['literacy-group'],'ratio-males':x['males-ratio-of-1'],"literacy-group-females":y['literacy-group'],'ratio-females':y['females-ratio-of-1']}, index=[0]))

In [72]:
final_df.to_csv(r'literacy-gender-c.csv', index = None, header=True) # WRITING OUR DATAFRAME INTO OUTPUT CSV FILE